In [194]:
import pandas as pd
import os 

# 1.TRAINING 

## 1.1. Gathering data's 

In [195]:
os.chdir('/Users/constancevandendorpe/Desktop/UCLM2Q1/LBIRTI2101B - Data Science in bioscience enginerring - Partim B /datasets')
Data_entrainement = pd.read_csv('./training.1600000.processed.noemoticon.csv',header = None,encoding = 'ISO-8859-1')

In [196]:
# Affichage du dataset d'entrainement 
Data_entrainement.columns = ['Polarity', 'Tweet_ID','Date','Query_username','Username','Tweet_text']
Data_entrainement

,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [200]:
# Besoin d'entrainer sur beaucoup moins de tweets (j'en prends que 2n ici, n positifs et n négatifs)! 
n = 5000
number_tweets = list(Data_entrainement.shape)[0]
print('On a',number_tweets, 'de tweets au total. On va donc seulement en prendre',2*n,'.')

Datapos = Data_entrainement[Data_entrainement['Polarity']==0]
Dataneg = Data_entrainement[Data_entrainement['Polarity']==4]

Data_sub = pd.concat([Datapos.sample(n),Dataneg.sample(n)], axis=0)
Data_sub.head()

On a 1600000 de tweets au total. On va donc seulement en prendre 10000 .


,Polarity,Tweet_ID,Date,Query_username,Username,Tweet_text
479620,0,2178951876,Mon Jun 15 08:26:16 PDT 2009,NO_QUERY,becksdavis,I want to go to @SlowsBBQ! Everyone keeps talk...
301457,0,1998654666,Mon Jun 01 18:54:33 PDT 2009,NO_QUERY,fentonslee,@fyreflye17 @naomitripi and spouse seems to th...
770822,0,2302122925,Tue Jun 23 16:22:16 PDT 2009,NO_QUERY,mike_nelson,@ChristineMarieN Wow you guys have a great loo...
623374,0,2229530857,Thu Jun 18 15:51:13 PDT 2009,NO_QUERY,twins160,I hate how people grow apart But SUCH great ...
597075,0,2219177147,Wed Jun 17 23:57:57 PDT 2009,NO_QUERY,YGee,RIP Lou


## 1.2. Dividing between training and testing data's

In [201]:
from sklearn.model_selection import train_test_split

In [202]:
# On va diviser ce dataset en données d'entrainement et données test. J'ai choisi au hasard une proportion 70/30. 
Proportion_training = 0.7
Data_train, Data_test = train_test_split(Data_sub, test_size=(1-Proportion_training))

## 1.3. Vectorizing the data's

In [231]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Pas de parametres particuliers à inclure (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
vectorizer = TfidfVectorizer()

train_vectors = vectorizer.fit_transform(Data_train['Tweet_text'])
test_vectors = vectorizer.transform(Data_test['Tweet_text'])

Interpreting the output of vectorizer : 

Assume general form: (A,B) C 

A: Document index B: Specific word-vector index C: TFIDF score for word B in document A

This is a sparse matrix. It indicates the tfidf score for all non-zero values in the word vector for each document.

In [232]:
print('Ce qui sort du vectorizer est une matrice de taille',train_vectors.shape,', où chaque ligne correspond à un document et chaque colonne correspond à un mot.\n Chaque case de la matrice est un score TFIDF. La matrice contient donc plein de 0, c est pour cela qu on imprime que les couples (document,mot) pour lesquels il y a un score.\n')
print(train_vectors)

Ce qui sort du vectorizer est une matrice de taille (6999, 14646) , où chaque ligne correspond à un document et chaque colonne correspond à un mot.
 Chaque case de la matrice est un score TFIDF. La matrice contient donc plein de 0, c est pour cela qu on imprime que les couples (document,mot) pour lesquels il y a un score.

  (0, 6488)	0.11395623588725694
  (0, 6417)	0.2419592188529535
  (0, 4015)	0.3283018952532073
  (0, 6920)	0.14394211906678459
  (0, 11844)	0.24278192593183476
  (0, 8154)	0.22884313640951345
  (0, 9375)	0.20091253618342947
  (0, 5820)	0.3283018952532073
  (0, 12766)	0.09402055181811271
  (0, 14158)	0.1476257691682406
  (0, 9432)	0.16045371869732852
  (0, 2181)	0.31658672887706185
  (0, 13186)	0.35652866125989663
  (0, 11838)	0.1818099318139314
  (0, 12406)	0.22884313640951345
  (0, 875)	0.17946163606265214
  (0, 6596)	0.3730402608904405
  (1, 9811)	0.3824548199262196
  (1, 913)	0.18854988605556103
  (1, 3966)	0.3245765486403304
  (1, 1223)	0.15151647823928938
  (1, 1

## 1.4. Creating a Linear SVM Model

In [238]:
import time
from sklearn import svm
from sklearn.metrics import classification_report

# Perform classification with SVM, kernel=linear

# Le classifier sera SVC : take as input two arrays: an array X of shape (n_samples, n_features) 
# holding the training samples, and an array y of class labels (strings or integers), of shape (n_samples)

# Training 
classifier_linear = svm.SVC(kernel='linear') 
classifier_linear.fit(train_vectors, Data_train['Polarity']) #fit pour définir le modèle 

# Testing 
prediction_linear = classifier_linear.predict(test_vectors) #predict pour prédire 

# results
#report = classification_report(Data_test['Polarity'], prediction_linear, output_dict=True)
#print('positive: ', report['0'])
#print('negative: ', report['4'])
print('prediction_linear est une liste qui contient des 0 et des 4.')
print('La précision pour les tweets positifs est de',report['0']['precision'],'\n La précision pour les tweets négatifs est de',report['4']['precision'])

prediction_linear est une liste qui contient des 0 et des 4.
La précision pour les tweets positifs est de 0.7340894770006301 
 La précision pour les tweets négatifs est de 0.743988684582744


# 2. US ELECTIONS

## 2.1. Gathering datas 

In [239]:
Data_trump = pd.read_csv('./hashtag_donaldtrump.csv',delimiter=',',engine = 'python')
Data_biden = pd.read_csv('./hashtag_joebiden.csv',delimiter=',')

In [192]:
# Quelle est la taille de ces datasets ? 
print('Trumps dataset is', list(Data_trump.shape)[0], 'long.')
print('Bidens dataset is', list(Data_biden.shape)[0], 'long.')

Trumps dataset is 331899 long.
Bidens dataset is 82348 long.


In [240]:
# On ne va garder que les colonnes sur les tweets et leur id
# Besoin d'entrainer sur beaucoup moins de tweets (j'en prends que 10 000 ici, 5000 pour Biden et 5000 pour Trump)! 

Data_biden_small = Data_biden[['tweet_id','tweet']].sample(5000)
Data_trump_small = Data_trump[['tweet_id','tweet']].sample(5000)

## 2.2. Preprocessing and vectorizing 

In [241]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import warnings
warnings.filterwarnings('ignore')

A propos du prepro : Il y a un preprocesssing plus important à faire sur les datasets trump et biden que sur le dataset d'entrainement. 
- Les étapes de text-cleaning & lowercase sont faites ici. 
- L'étape de tokenization est faite avec scikitlearn. 
- Les étapes de removal des stop words et stemming ne sont pas réalisées. 

In [243]:
def prepro_SVM(text,i):
    clean=text[i].astype(str).str.replace("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ")
    #lowercase=clean.str.lower()
    #words=lowercase.apply(word_tokenize)
    #stop_words = stopwords.words('english') 
    #filtered_words = words.apply(lambda x:[word for word in x if word not in stop_words])
    #porter = PorterStemmer()
    #text['Tweet_clean']=filtered_words.apply(lambda x:[porter.stem(word) for word in x])
    text['Tweet_clean']=clean.str.lower()
    return text

In [244]:
Data_biden_clean=prepro_SVM(Data_biden_small,'tweet')
Data_trump_clean=prepro_SVM(Data_trump_small,'tweet')

In [245]:
# Vectorization en utilisant le même vectorizer que pour le dataset d'entrainement, 
# après avoir un peu nettoyé les données. 
vectors_trump = vectorizer.transform(Data_trump_clean['Tweet_clean'])
vectors_biden = vectorizer.transform(Data_biden_clean['Tweet_clean'])

## 2.3. Predicting 

In [246]:
# On utilise le classifier généré pendant la phase d'entrainement
prediction_trump = classifier_linear.predict(vectors_trump)
prediction_biden = classifier_linear.predict(vectors_biden)

In [247]:
trump_list = list(prediction_trump)
biden_list = list(prediction_biden)
print('Trump has a positive percentage of :', trump_list.count(4)/len(trump_list)*100,'%')
print('Biden has a positive percentage of :', biden_list.count(4)/len(biden_list)*100,'%')

Trump has a positive percentage of : 63.0 %
Biden has a positive percentage of : 65.03999999999999 %
